In [1]:
# 加载需要的包
import pandas as pd
import numpy as np
import datetime
from typing import Union

In [2]:
# 连接wind
from WindPy import w
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
def convert_dates(date: Union[None, str, datetime.date]) -> datetime.date:
    """
    convert different types of date data into datetime.date
    :param date: can be None, string, datetime.date, QuantLib.Date
    :return: datetime.date
    """
    if date is None:
        today = datetime.date.today()
        return datetime.date(today.year + 100, today.month, today.day)
    elif isinstance(date, str):
        return datetime.datetime.strptime(date, "%Y-%m-%d").date()
    elif isinstance(date, datetime.datetime):
        return date.date()
    elif isinstance(date, datetime.date):
        return date
    else:
        raise Exception(f"Unrecognized date {date} with {type(date)} !!")

In [4]:
def get_dates_between_period(
        start_date: Union[str, datetime.date],
        end_date: Union[str, datetime.date],
        freq: str = 'D'
) -> list:
        date_range = pd.date_range(start_date, end_date, freq=freq).to_list()
        date_range = [convert_dates(i) for i in date_range]

        return date_range



In [5]:
# 参数设置
trade_date = '20240630'
start_date = '2021-09-30'
end_date = '2024-06-30' 
quarters = get_dates_between_period(start_date,end_date,freq='3M')

In [6]:
# 数据读取
fof_data = pd.read_excel('./data/基金代码.xlsx').fillna(0)
fund_code = fof_data['证券代码'].to_list()

2.产品规模权重

In [72]:
# 从Wind中获取基金公司和业绩数据
fund_return = w.wss(fund_code,"return_1y, return_2y, return_3y","annualized=0;tradeDate=" + trade_date).Data
fund_mgrcomp = w.wss(fund_code, "fund_mgrcomp, fund_setupdate, sec_name").Data
return_df = pd.DataFrame({'基金代码': fund_code, '基金简称':fund_mgrcomp[2],'成立日期':fund_mgrcomp[1], '基金公司': fund_mgrcomp[0],'近一年回报': fund_return[0], '近两年回报': fund_return[1], '近三年回报': fund_return[2]})

In [73]:
return_df = return_df.drop(index=0)

In [74]:
# 从Wind中获取规模数据
scale_data = {}
scale_data['基金代码'] = fund_code
scale_data['基金简称'] = fund_mgrcomp[2]
scale_data['基金公司'] = fund_mgrcomp[0]
scale_data['成立日期'] = fund_mgrcomp[1]
for date in quarters:
    wind_data = w.wss(fund_code, "netasset_total", f"unit=1;tradeDate={date.strftime('%Y-%m-%d')};").Data[0]
    wind_data = [round(x / 1e8, 2) if isinstance(x, (int, float)) and x is not None else x for x in wind_data]
    scale_data[date.strftime('%Y%m%d')] = wind_data

In [75]:
fund_scale_df = pd.DataFrame(scale_data)

In [76]:
fund_scale_df

,基金代码,基金简称,基金公司,成立日期,20210930,20211231,20220331,20220630,20220930,20221231,20230331,20230630,20230930,20231231,20240331,20240630
0,0,None,None,1899-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,005156.OF,嘉实领航资产配置A,嘉实基金管理有限公司,2017-10-26,1.83,1.81,1.62,1.60,1.49,1.43,1.45,1.39,1.28,1.24,22.25,50.77
2,005215.OF,南方全天候策略A,南方基金管理股份有限公司,2017-10-19,39.82,46.22,39.71,35.42,32.39,29.63,28.31,26.26,22.14,18.93,17.27,17.01
3,005217.OF,建信福泽安泰A,建信基金管理有限责任公司,2017-11-02,1.36,1.36,1.20,1.18,1.09,1.05,1.04,0.97,0.92,0.85,0.83,0.80
4,005218.OF,华夏聚惠稳健目标A,华夏基金管理有限公司,2017-11-03,2.55,2.86,2.71,2.50,2.40,2.24,2.14,2.02,1.94,1.87,1.82,1.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,900012.OF,中信证券财富优选一年持有A,中信证券资产管理有限公司,2021-07-22,0.60,10.70,11.16,11.77,10.49,9.82,9.10,7.90,6.81,6.17,5.58,5.27
522,952013.OF,国泰君安君得益三个月持有A,上海国泰君安证券资产管理有限公司,2020-11-16,36.23,27.38,21.39,20.67,17.40,16.90,16.75,15.29,13.53,12.04,11.19,10.68
523,970138.OF,银河安益9个月持有A,银河金汇证券资产管理有限公司,2022-03-02,NaN,NaN,NaN,0.58,0.52,0.50,0.47,0.54,0.52,0.48,0.54,0.54
524,970194.OF,兴证资管金麒麟3个月A,兴证证券资产管理有限公司,2022-09-05,NaN,NaN,NaN,NaN,NaN,2.34,1.52,1.30,1.18,1.09,1.02,0.95


In [77]:
Range_of_return = pd.DataFrame(scale_data)

In [78]:
fund_scale_df['近三年平均规模'] = fund_scale_df.iloc[:, 2:14].mean(axis=1)
fund_scale_df['近两年平均规模'] = fund_scale_df.iloc[:, 6:14].mean(axis=1)
fund_scale_df['近一年平均规模'] = fund_scale_df.iloc[:, 10:14].mean(axis=1)
index_1 = fund_scale_df.loc[fund_scale_df.iloc[:, 2:14].isnull().any(axis=1)].index
index_2 = fund_scale_df.loc[fund_scale_df.iloc[:, 6:14].isnull().any(axis=1)].index
index_3 = fund_scale_df.loc[fund_scale_df.iloc[:, 10:14].isnull().any(axis=1)].index
fund_scale_df['近三年平均规模'][index_1] = np.nan
fund_scale_df['近两年平均规模'][index_2] = np.nan
fund_scale_df['近一年平均规模'][index_3] = np.nan

fund_scale_df.insert(4, '近三年平均规模', fund_scale_df.pop('近三年平均规模'))
fund_scale_df.insert(5, '近两年平均规模', fund_scale_df.pop('近两年平均规模'))
fund_scale_df.insert(6, '近一年平均规模', fund_scale_df.pop('近一年平均规模'))

<ipython-input-78-5ba89c3e95d2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fund_scale_df['近三年平均规模'][index_1] = np.nan
<ipython-input-78-5ba89c3e95d2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fund_scale_df['近两年平均规模'][index_2] = np.nan
<ipython-input-78-5ba89c3e95d2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fund_scale_df['近一年平均规模'][index_3] = np.nan


In [79]:
grouped = fund_scale_df.groupby('基金公司').agg({'近三年平均规模': 'sum', '近两年平均规模': 'sum', '近一年平均规模': 'sum'}).reset_index()
grouped.columns = ['基金公司', '近三年总规模', '近两年总规模', '近一年总规模', ]
fund_scale_df = fund_scale_df.merge(grouped, on='基金公司')

In [80]:
fund_scale_df['近三年规模权重'] = fund_scale_df['近三年平均规模'] / fund_scale_df['近三年总规模']
fund_scale_df['近两年规模权重'] = fund_scale_df['近两年平均规模'] / fund_scale_df['近两年总规模']
fund_scale_df['近一年规模权重'] = fund_scale_df['近一年平均规模'] / fund_scale_df['近一年总规模']

fund_scale_df.insert(7, '近三年总规模', fund_scale_df.pop('近三年总规模'))
fund_scale_df.insert(8, '近两年总规模', fund_scale_df.pop('近两年总规模'))
fund_scale_df.insert(9, '近一年总规模', fund_scale_df.pop('近一年总规模'))
fund_scale_df.insert(4, '近三年规模权重', fund_scale_df.pop('近三年规模权重'))
fund_scale_df.insert(5, '近两年规模权重', fund_scale_df.pop('近两年规模权重'))
fund_scale_df.insert(6, '近一年规模权重', fund_scale_df.pop('近一年规模权重'))

In [83]:
fund_scale_df.insert(2,'成立日期',fund_scale_df.pop('成立日期'))

In [84]:
fund_scale_df

,基金代码,基金简称,成立日期,基金公司,近三年规模权重,近两年规模权重,近一年规模权重,近三年平均规模,近两年平均规模,近一年平均规模,...,20220331,20220630,20220930,20221231,20230331,20230630,20230930,20231231,20240331,20240630
0,005156.OF,嘉实领航资产配置A,2017-10-26,嘉实基金管理有限公司,0.038512,0.036811,0.039121,1.514,1.43750,1.3400,...,1.62,1.60,1.49,1.43,1.45,1.39,1.28,1.24,22.25,50.77
1,006245.OF,嘉实养老2030三年A,2019-08-05,嘉实基金管理有限公司,0.071810,0.065043,0.062039,2.823,2.54000,2.1250,...,3.81,4.06,1.95,2.00,2.26,2.24,2.04,1.96,1.93,1.87
2,006307.OF,嘉实养老2040五年A,2019-03-06,嘉实基金管理有限公司,0.063721,0.065555,0.078753,2.505,2.56000,2.6975,...,2.29,2.51,2.38,2.51,2.78,2.77,2.64,2.60,2.44,2.40
3,007188.OF,嘉实养老2050五年A,2019-04-25,嘉实基金管理有限公司,0.062042,0.072597,0.089847,2.439,2.83500,3.0775,...,2.35,2.71,2.54,2.77,3.13,3.14,3.00,3.04,3.15,3.02
4,010277.OF,嘉实民安添岁稳健养老一年持有A,2021-01-13,嘉实基金管理有限公司,0.761396,0.601805,0.508211,29.932,23.50125,17.4075,...,35.81,32.53,26.23,23.81,20.41,18.16,16.26,14.80,14.19,13.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,870004.OF,广发资管智荟广易六个月持有A,2022-04-11,广发证券资产管理(广东)有限公司,NaN,NaN,1.000000,NaN,NaN,1.4550,...,NaN,1.06,1.67,1.63,1.58,1.49,1.41,1.34,1.29,1.25
521,900012.OF,中信证券财富优选一年持有A,2021-07-22,中信证券资产管理有限公司,1.000000,1.000000,1.000000,8.452,9.15250,7.4950,...,11.16,11.77,10.49,9.82,9.10,7.90,6.81,6.17,5.58,5.27
522,970138.OF,银河安益9个月持有A,2022-03-02,银河金汇证券资产管理有限公司,NaN,NaN,1.000000,NaN,NaN,0.5025,...,NaN,0.58,0.52,0.50,0.47,0.54,0.52,0.48,0.54,0.54
523,970194.OF,兴证资管金麒麟3个月A,2022-09-05,兴证证券资产管理有限公司,NaN,NaN,1.000000,NaN,NaN,1.2725,...,NaN,NaN,NaN,2.34,1.52,1.30,1.18,1.09,1.02,0.95


3.加权计算

In [85]:
fund_performance = fund_scale_df.merge(return_df, on='基金代码')
fund_performance['近三年加权回报'] = fund_performance['近三年规模权重'] * fund_performance['近三年回报']
fund_performance['近两年加权回报'] = fund_performance['近两年规模权重'] * fund_performance['近两年回报']
fund_performance['近一年加权回报'] = fund_performance['近一年规模权重'] * fund_performance['近一年回报']

In [86]:
fund_performance.drop(['近三年规模权重', '近两年规模权重', '基金简称_y', '成立日期_y', '近一年规模权重', '近三年回报', '近两年回报', '近一年回报'] ,axis=1, inplace=True)

In [87]:
fund_performance.drop(['近三年总规模','近两年总规模','近一年总规模','基金公司_y','近三年平均规模','近两年平均规模','近一年平均规模','20210930','20211231','20220331','20220630','20220930','20221231','20230331','20230630','20230930','20231231','20240331','20240630'],axis=1,inplace=True)

In [88]:
fund_performance.reset_index()

,index,基金代码,基金简称_x,成立日期_x,基金公司_x,近三年加权回报,近两年加权回报,近一年加权回报
0,0,005156.OF,嘉实领航资产配置A,2017-10-26,嘉实基金管理有限公司,-0.463615,-0.356149,-0.162326
1,1,006245.OF,嘉实养老2030三年A,2019-08-05,嘉实基金管理有限公司,-1.199524,-1.236584,-0.686639
2,2,006307.OF,嘉实养老2040五年A,2019-03-06,嘉实基金管理有限公司,-1.379787,-1.471666,-1.083788
3,3,007188.OF,嘉实养老2050五年A,2019-04-25,嘉实基金管理有限公司,-1.385396,-1.697656,-1.254128
4,4,010277.OF,嘉实民安添岁稳健养老一年持有A,2021-01-13,嘉实基金管理有限公司,-2.608347,-2.687789,-1.081083
...,...,...,...,...,...,...,...,...
520,520,870004.OF,广发资管智荟广易六个月持有A,2022-04-11,广发证券资产管理(广东)有限公司,NaN,NaN,-2.595724
521,521,900012.OF,中信证券财富优选一年持有A,2021-07-22,中信证券资产管理有限公司,NaN,-25.601313,-13.601326
522,522,970138.OF,银河安益9个月持有A,2022-03-02,银河金汇证券资产管理有限公司,NaN,NaN,0.476948
523,523,970194.OF,兴证资管金麒麟3个月A,2022-09-05,兴证证券资产管理有限公司,NaN,NaN,-15.559909


In [89]:
def sum_for_nan(series):
    if series.count() == 0:
        sum_result = np.nan
    else:
        sum_result = series.sum()
    return sum_result

In [90]:
mgrcomp_list = grouped['基金公司'].tolist()
mgrcomp_rank = pd.DataFrame()
for mgrcomp in mgrcomp_list:
    sum_3y = sum_for_nan(fund_performance['近三年加权回报'][fund_performance['基金公司_x'] == mgrcomp])
    sum_2y = sum_for_nan(fund_performance['近两年加权回报'][fund_performance['基金公司_x'] == mgrcomp])
    sum_1y = sum_for_nan(fund_performance['近一年加权回报'][fund_performance['基金公司_x'] == mgrcomp])
    new_row = pd.DataFrame([{'基金公司': mgrcomp, '近三年收益率': sum_3y, '近两年收益率': sum_2y, '近一年收益率': sum_1y, }])
    mgrcomp_rank = pd.concat([mgrcomp_rank, new_row])


In [91]:
mgrcomp_rank['近三年排名'] = mgrcomp_rank['近三年收益率'].rank(method='min', ascending=False)
mgrcomp_rank['近两年排名'] = mgrcomp_rank['近两年收益率'].rank(method='min', ascending=False)
mgrcomp_rank['近一年排名'] = mgrcomp_rank['近一年收益率'].rank(method='min', ascending=False)

In [92]:
mgrcomp_rank.insert(2,'近三年排名',mgrcomp_rank.pop('近三年排名'))
mgrcomp_rank.insert(4,'近两年排名',mgrcomp_rank.pop('近两年排名'))

In [93]:
threshold_3 = mgrcomp_rank['近三年排名'].max() * 0.7
threshold_2 = mgrcomp_rank['近两年排名'].max() * 0.7
threshold_1 = mgrcomp_rank['近一年排名'].max() * 0.7

def calculate_score(row):
    if pd.isna(row['近三年排名']) and pd.isna(row['近两年排名']):
        return '是' if row['近一年排名'] > threshold_1 else ''
    elif pd.isna(row['近三年排名']):
        return '是' if (row['近两年排名'] > threshold_2) & (row['近一年排名'] > threshold_1) else ''
    else:
        return '是' if (row['近三年排名'] > threshold_3) & (row['近两年排名'] > threshold_2) & (row['近一年排名'] > threshold_1) else ''

# 添加"得分是否在后30%"列
mgrcomp_rank['业绩排名是否在后30%'] = mgrcomp_rank.apply(calculate_score, axis=1)

In [94]:
mgrcomp_rank=mgrcomp_rank.sort_values(by='近一年排名', ascending=True)

In [95]:
mgrcomp_rank

,基金公司,近三年收益率,近三年排名,近两年收益率,近两年排名,近一年收益率,近一年排名,业绩排名是否在后30%
0,财通证券资产管理有限公司,NaN,NaN,NaN,NaN,2.735569,1.0,
0,金鹰基金管理有限公司,NaN,NaN,NaN,NaN,2.491744,2.0,
0,浦银安盛基金管理有限公司,0.329943,3.0,-2.144861,4.0,2.005645,3.0,
0,浙商基金管理有限公司,NaN,NaN,NaN,NaN,1.692369,4.0,
0,永赢基金管理有限公司,NaN,NaN,1.963294,1.0,0.523531,5.0,
...,...,...,...,...,...,...,...,...
0,国联安基金管理有限公司,NaN,NaN,NaN,NaN,NaN,NaN,
0,太平基金管理有限公司,NaN,NaN,NaN,NaN,NaN,NaN,
0,汇丰晋信基金管理有限公司,NaN,NaN,NaN,NaN,NaN,NaN,
0,渤海汇金证券资产管理有限公司,NaN,NaN,NaN,NaN,NaN,NaN,


In [96]:
df2 = fund_performance.groupby('基金公司_x').agg(
    近三年总和=('近三年加权回报', 'sum'),
    近两年总和=('近两年加权回报', 'sum'),
    近一年总和=('近一年加权回报', 'sum')
)
df2.insert(0, '近三年收益率', df2.pop('近三年总和'))
df2.insert(1, '近两年收益率', df2.pop('近两年总和'))
df2.insert(2, '近一年收益率', df2.pop('近一年总和'))
df2.index.name = '基金公司'

df2_filtered = fund_scale_df[['基金公司', '近三年总规模', '近两年总规模', '近一年总规模']].drop_duplicates(subset='基金公司')

# 合并数据，根据基金公司名称进行合并
result_df = pd.merge(df2, df2_filtered, on='基金公司', how='left')
result_df.insert(2, '近三年规模', result_df.pop('近三年总规模'))
result_df.insert(4, '近两年规模', result_df.pop('近两年总规模'))
result_df.insert(6, '近一年规模', result_df.pop('近一年总规模'))

result_df['收益率百分位3'] = result_df['近三年收益率'].rank(pct=True)
result_df['收益率百分位2'] = result_df['近两年收益率'].rank(pct=True)
result_df['收益率百分位1'] = result_df['近一年收益率'].rank(pct=True)

result_df['规模百分位3'] = result_df['近三年规模'].rank(pct=True)
result_df['规模百分位2'] = result_df['近两年规模'].rank(pct=True)
result_df['规模百分位1'] = result_df['近一年规模'].rank(pct=True)

# 计算得分（收益率百分位和规模百分位的平均值）
result_df['得分3'] = (result_df['收益率百分位3'] + result_df['规模百分位3']) / 2
result_df['得分2'] = (result_df['收益率百分位2'] + result_df['规模百分位2']) / 2
result_df['得分1'] = (result_df['收益率百分位1'] + result_df['规模百分位1']) / 2

# 根据得分进行排名
result_df['排名3'] = result_df['得分3'].rank(ascending=False, method='min')
result_df['排名2'] = result_df['得分2'].rank(ascending=False, method='min')
result_df['排名1'] = result_df['得分1'].rank(ascending=False, method='min')

result_df.insert(3,'得分3',result_df.pop('得分3'))
result_df.insert(4,'排名3',result_df.pop('排名3'))
result_df.insert(7,'得分2',result_df.pop('得分2'))
result_df.insert(8,'排名2',result_df.pop('排名2'))
result_df.insert(11,'得分1',result_df.pop('得分1'))
result_df.insert(12,'排名1',result_df.pop('排名1'))

# 指定要删除的列
columns_to_drop = ['收益率百分位3','收益率百分位2','收益率百分位1','规模百分位3','规模百分位2','规模百分位1']

# 删除指定列
result_df = result_df.drop(columns=columns_to_drop)

# 计算总行数和阈值
threshold_E = result_df['排名3'].max() * 0.7
threshold_I = result_df['排名2'].max() * 0.7
threshold_M = result_df['排名1'].max() * 0.7

# 定义一个函数来应用复杂的条件逻辑
def calculate_score(row):
    if pd.isna(row['排名3']) and pd.isna(row['排名2']):
        return '是' if row['排名1'] > threshold_M else ''
    elif pd.isna(row['排名3']):
        return '是' if (row['排名2'] > threshold_I) & (row['排名1'] > threshold_M) else ''
    else:
        return '是' if (row['排名3'] > threshold_E) & (row['排名2'] > threshold_I) & (row['排名1'] > threshold_M) else ''

# 添加"得分是否在后30%"列
result_df['得分是否在后30%'] = result_df.apply(calculate_score, axis=1)

# 计算每列的数量
count_3 = len(result_df['排名3'])
count_2 = len(result_df['排名2'])
count_1 = len(result_df['排名1'])

# 创建一个 DataFrame 来包含这些数量
count_df = pd.DataFrame({
    '基金公司_x': ['总数'],
    '排名3': [count_3],
    '排名2': [count_2],
    '排名1': [count_1]
})

# 将新的行添加到 DataFrame 的末尾
result_df = pd.concat([result_df, count_df], ignore_index=True)

result_df = result_df.sort_values(by='排名1', ascending=True)
result_df.reset_index()

,index,基金公司,近三年收益率,近三年规模,得分3,排名3,近两年收益率,近两年规模,得分2,排名2,近一年收益率,近一年规模,得分1,排名1,得分是否在后30%,基金公司_x
0,70,浦银安盛基金管理有限公司,0.329943,66.879,0.960674,1.0,-2.144861,79.35750,0.797753,1.0,2.005645,53.6250,0.932584,1.0,,NaN
1,31,南方基金管理股份有限公司,-6.483132,98.460,0.668539,11.0,-5.879200,91.22250,0.741573,4.0,-2.564319,99.4425,0.820225,2.0,,NaN
2,16,交银施罗德基金管理有限公司,-0.674875,222.203,0.758427,3.0,-5.236581,229.95375,0.780899,2.0,-3.664628,170.3600,0.780899,3.0,,NaN
3,79,金鹰基金管理有限公司,0.000000,0.000,0.469101,37.0,0.000000,0.00000,0.500000,34.0,2.491744,4.0900,0.769663,4.0,,NaN
4,56,招商基金管理有限公司,1.303766,31.263,0.926966,2.0,-2.301189,33.92625,0.747191,3.0,-2.450808,31.1650,0.764045,5.0,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,14,中银国际证券股份有限公司,0.000000,0.000,0.469101,37.0,0.000000,0.00000,0.500000,34.0,-12.396689,1.3800,0.207865,85.0,,NaN
86,20,兴证证券资产管理有限公司,0.000000,0.000,0.469101,37.0,0.000000,0.00000,0.500000,34.0,-15.559909,1.2725,0.174157,87.0,,NaN
87,75,西藏东财基金管理有限公司,0.000000,0.000,0.469101,37.0,0.000000,0.00000,0.500000,34.0,-11.695183,0.4100,0.140449,88.0,,NaN
88,76,西部利得基金管理有限公司,0.000000,0.000,0.469101,37.0,0.000000,0.00000,0.500000,34.0,-12.351747,0.2675,0.123596,89.0,,NaN


In [97]:
result_df.pop('基金公司_x')

70    NaN
31    NaN
16    NaN
79    NaN
56    NaN
     ... 
14    NaN
20    NaN
75    NaN
76    NaN
89     总数
Name: 基金公司_x, Length: 90, dtype: object

In [98]:
filtered_df1 = mgrcomp_rank[mgrcomp_rank['业绩排名是否在后30%'] == '是']
filtered_df2 = result_df[result_df['得分是否在后30%'] == '是']

# 提取基金公司名称
companies_df1 = filtered_df1['基金公司']
companies_df2 = filtered_df2['基金公司']
contrast_df = pd.DataFrame({
    '业绩排名都在后30%的基金公司': companies_df1.reset_index(drop=True),
    '综合得分都在后30%的基金公司': companies_df2.reset_index(drop=True)
})

In [99]:
df3 = pd.read_excel('./data/基金公司.xlsx', sheet_name='Sheet1')

def compare_companies(row):
    if row['基金公司'] in contrast_df['业绩排名都在后30%的基金公司'].values:
        row['与加权业绩排名比对'] = '包含'
    else:
        row['与加权业绩排名比对'] = '不包含'
    if row['基金公司'] in contrast_df['综合得分都在后30%的基金公司'].values:
        row['与综合得分对比'] = '包含'
    else:
        row['与综合得分对比'] = '不包含'
    return row

def get_ranking(row):
    matches = result_df.loc[result_df['基金公司'] == row['基金公司'], '排名1']
    if not matches.empty:
        row['近一年综合得分排名'] = matches.values[0]
    else:
        row['近一年综合得分排名'] = '不包含'  # 或者你希望的其他默认值
    return row

df3 = df3.apply(compare_companies, axis=1)
df3 = df3.apply(get_ranking, axis=1)

In [105]:
from openpyxl import load_workbook
df_dict = {}

def excel_output_process(df_dict: dict, input_path: str, output_path: str):
    template_excel = load_workbook(input_path)
    
    col_offsets = [0, 0, 0, 0, 0, 0, 0]  # 定义列偏移量
    i = 0
    for sheet_name, df in df_dict.items():
        # 重置索引
        df = df.reset_index(drop=True)
        
        # 获取目标工作表对象
        sheet = template_excel[sheet_name]
        
        for r_idx, row in enumerate(df.values, start=2):
            for c_idx, value in enumerate(row, start=1):
                # 计算列偏移量
                col_offset = col_offsets[(c_idx - 1) % len(col_offsets)]
                # 动态调整列索引
                sheet.cell(row=r_idx + 1, column=c_idx + col_offsets[i], value=value)
        i = i + 1
    # 保存修改后的 Excel 文件
    template_excel.save(output_path)

In [107]:
df_dict['1.产品区间收益率'] = return_df
df_dict['2.产品规模权重'] = fund_scale_df
df_dict['3.加权计算过程'] = fund_performance
df_dict['4.加权业绩排名'] = mgrcomp_rank
df_dict['5.综合得分'] = result_df
df_dict['6.低分名单'] = contrast_df
df_dict['7.名单比对'] = df3

In [108]:
excel_output_process(df_dict, './data/fof基金模版.xlsx', './output/FOF基金排名.xlsx')